In [1]:
import pandas as pd

In [2]:
# Read BART.far data into DataFrame
df = pd.read_table('BART.far', header=None, names=['node_from', 'node_to', 
                                                  'fare', ';', 'name_from', 'name_to'])
df = df.drop(';', axis=1)
df.name_from = df.name_from.str.replace(" to", "")

In [3]:
# get unique node number and name:
lookup = {}
for x in list(set(list(df[['node_from','name_from']].apply(lambda x: str(x[0]) + '@' + x[1], axis=1).unique()) + list(df[['node_to','name_to']].apply(lambda x: str(x[0]) + '@' + x[1], axis=1).unique()))):
    lookup[x.split('@')[1]] = int(x.split('@')[0])

lookup['Ballpark'] = 18001
lookup['2nd/Mission'] = 18002
lookup['Union Square'] = 18003
lookup['Van Ness'] = 18004
lookup['Fillmore'] = 18005
lookup['Jack London Square'] = 18006 

TypeError: ("Can't convert 'float' object to str implicitly", 'occurred at index 1378')

In [16]:
# df_new = pd.DataFrame.from_csv('BART_TestRunNewStationFare.csv')
# df_new['node_from'] = df_new.index
# df_new.reset_index(drop=True,inplace=True)
# df_new = df_new[['node_from','node_to','fare','name_from','name_to']]
# df_new.head()

,node_from,node_to,fare,name_from,name_to
0,18006,18001,217,JLS,BALLPARK
1,18006,18002,217,JLS,2ND AND MISSION
2,18006,18003,217,JLS,UNION SQUARE
3,18006,18004,217,JLS,VANNESS
4,18006,18005,217,JLS,FILLMORE


In [17]:
# df_new['name_from'] = df_new['name_from'].str.replace('FRUITVALE', 'Fruitvale')
# df_new['name_to'] = df_new['name_to'].str.replace('FRUITVALE', 'Fruitvale')
# df_new['name_from'] = df_new['name_from'].str.replace('12OAK', '12th St Oakland City Center')
# df_new['name_to'] = df_new['name_to'].str.replace('12OAK', '12th St Oakland City Center')
# df_new

,node_from,node_to,fare,name_from,name_to
0,18006,18001,217,JLS,BALLPARK
1,18006,18002,217,JLS,2ND AND MISSION
2,18006,18003,217,JLS,UNION SQUARE
3,18006,18004,217,JLS,VANNESS
4,18006,18005,217,JLS,FILLMORE
5,18006,15533,129,JLS,Fruitvale
6,18006,15509,129,JLS,12th St Oakland City Center
7,18001,18002,129,BALLPARK,2ND AND MISSION
8,18001,18003,129,BALLPARK,UNION SQUARE
9,18001,18004,129,BALLPARK,VANNESS


In [22]:
# Make DataFrame of new nodes with additive fares to each proxy station. 
# We will take fares from each proxy station to each other station in the BART network,
# Add the fare from the new station of interest (e.g. fare from Ballpark to Fruitvale)
# And find the minimum fare to each other station in the network

# Set additive fares below:

new_nodes = {
    'name':['Ballpark', '2nd/Mission', 'Union Square', 'Van Ness', 'Fillmore', 'Jack London Square'],
    'Fruitvale':[346, 346, 346, 346, 346, 129],
    '12th St Oakland City Center': [346, 346, 346, 346, 346, 129]
}
testnodes = pd.DataFrame(new_nodes)
testnodes.set_index('name', inplace=True)

In [23]:
testnodes

,12th St Oakland City Center,Fruitvale
name,,
Ballpark,346,346
2nd/Mission,346,346
Union Square,346,346
Van Ness,346,346
Fillmore,346,346
Jack London Square,129,129


In [4]:
def get_fares_from(df, name, add_fare):
    """
    This function gets all the fares from one station to every other in the original fare table.
    
    df : pandas DataFrame
        The original fare table
    name : str
        The name of the station that you want to get fares from
    add_fare : int
        Additional fare to add to each value in the existing fare table
        
    Returns a pandas Series of stations and fares
    """
    station = df.loc[(df.name_from == name) | (df.name_to == name)]
    station['destination'] = ''
    station.loc[station.name_from != name, 'destination'] = station.loc[station.name_from != name, 'name_from']
    station.loc[station.name_to != name, 'destination'] = station.loc[station.name_to != name, 'name_to']
    station.set_index('destination', inplace=True)
    
    series = pd.Series(data=(station.fare + add_fare), name=name)
    
    return series

In [5]:
fruitvale = get_fares_from(df, 'Fruitvale', 20)
fruitvale

/Users/yiyange/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/yiyange/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


destination
Dublin/ Pleasanton               300
West Dublin/Pleasanton           289
Castro Valley                    212
Fremont                          303
Union City                       279
South Hayward                    244
Hayward                          212
Bay Fair                         184
San Leandro                      149
Coliseum/Oakland Airport         149
Lake Merritt                     149
Richmond                         237
El Cerrito Del Norte             223
El Cerrito Plaza                 205
North Berkeley                   184
Berkeley                         174
Ashby                            163
Pittsburg/ Bay Point             352
North Concord                    317
Concord                          303
Pleasant Hill                    275
Walnut Creek                     268
Lafayette                        244
Orinda                           205
Rockridge                        153
MacArthur                        149
19th St Oakland           

In [28]:
ballpark = testnodes.loc['Ballpark']
ballpark

12th St Oakland City Center    346
Fruitvale                      346
Name: Ballpark, dtype: int64

In [29]:
def min_fare(series, fare_df):
    """
    Returns minimum fare to each station based on a series where index is the proxy station
    that we calculate all fares from, and value is the additive fare (e.g. from new stop to
    existing proxy station)
    """
    ser = pd.Series()
    for index, value in series.iteritems():
        fares = get_fares_from(df, index, value)
        ser = pd.concat([ser, fares], axis=1)

    fare = ser.min(axis=1)
    return fare

In [30]:
def get_all_fares(node_df, fare_df, output_df):
    """
    Takes in DataFrame of new nodes, with new station names as index, and proxy stations as columns,
    with additive fares to proxy stations as values in each column. Returns DataFrame in similar format to 
    original data. Need to add node IDs.
    """
    
    for index, row in node_df.iterrows():
        results = pd.DataFrame(min_fare(row, fare_df))
        results.reset_index(inplace=True)
        results.columns = ['name_to', 'fare']
        results['name_from'] = index
        output_df = pd.concat([output_df,results], axis = 0)
        
    return output_df
        

In [34]:
output_df = pd.DataFrame(columns=['name_from', 'name_to', 'fare'])
output = get_all_fares(testnodes, df, output_df)

/Users/yiyange/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/yiyange/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [40]:
output['node_from'] = output['name_from'].apply(lambda x: lookup[x])
output['node_to'] = output['name_to'].apply(lambda x: lookup[x])
output

,fare,name_from,name_to,node_from,node_to
0,475,Ballpark,12th St Oakland City Center,18001,15509
1,591,Ballpark,16th St Mission,18001,15515
2,475,Ballpark,19th St Oakland,18001,15508
3,598,Ballpark,24th St Mission,18001,15516
4,731,Ballpark,Alum Rock,18001,14204
5,475,Ballpark,Ashby,18001,15525
6,622,Ballpark,Balboa Park,18001,15518
7,510,Ballpark,Bay Fair,18001,15530
8,475,Ballpark,Berkeley,18001,15523
9,682,Ballpark,Berryessa,18001,14203


In [41]:
output = output[['node_from','node_to','fare', 'name_from', 'name_to']]

In [43]:
output.to_csv('NewFare.csv')